In [0]:
# Import libraries and modules
import tensorflow as tf
import numpy as np
import pandas as pd
import shutil
print tf.__version__
print np.__version__
print pd.__version__
np.set_printoptions(threshold=np.inf)

1.8.0
1.14.0
0.22.0


/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
# change these to try this notebook out
BUCKET = 'youtube8m-4-train'
PROJECT = 'qwiklabs-gcp-8d3d0cd07cef9252'
REGION = 'us-central1'

In [0]:
# Import os environment variables
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# Local Development

In [0]:
# Set logging verbosity to INFO for richer output
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
# The number of video classes
NUM_CLASSES = 4716
# The number of frames to extract per video
MAX_FRAMES = 20

In [0]:
arguments = {}
arguments["train_file_pattern"] = "gs://youtube-8m-team/1/frame_level/train/train*.tfrecord"
arguments["eval_file_pattern"] = "gs://youtube-8m-team/1/frame_level/validate/validate-0.tfrecord"
arguments["output_dir"] = "trained_model"
arguments["batch_size"] = 10
arguments["train_steps"] = 100
arguments["hidden_units"] = [1024, 256, 64]
arguments["top_k"] = 5
arguments["start_delay_secs"] = 60
arguments["throttle_secs"] = 30

In [0]:
# Create input function to read our training and validation data
def read_dataset_frame(file_pattern, mode, batch_size):
  def _input_fn():
    print("\nread_dataset_frame: _input_fn: file_pattern = {}".format(file_pattern))
    print("read_dataset_frame: _input_fn: mode = {}".format(mode))
    print("read_dataset_frame: _input_fn: batch_size = {}".format(batch_size))
    
    # This function dequantizes our tensors to bring them back to full floating point precision
    def dequantize(feat_vector, max_quantized_value = 2, min_quantized_value = -2):
      assert max_quantized_value > min_quantized_value # ensure the max value is larger than the min value
      quantized_range = max_quantized_value - min_quantized_value # find the range between max and min
      scalar = quantized_range / 255.0 # create a scale factor where 0 is the min and 1 is the max
      bias = (quantized_range / 512.0) + min_quantized_value # create bias term to shift our scaled feature vector
      return feat_vector * scalar + bias # return the scaled and shifted feature vector

    # This function resizes our frames axis so that we only get a subset of frames
    def resize_axis(tensor, axis, new_size, fill_value = 0):
      tensor = tf.convert_to_tensor(value = tensor) # ensure tensor is a tensor
      shape = tf.unstack(value = tf.shape(input = tensor)) # create a list where each element is a 1-D tensor the size of each dimension in tensor

      pad_shape = shape[:] # create a copy of the shape list of 1-D tensors
      pad_shape[axis] = tf.maximum(x = 0, y = new_size - shape[axis]) # change the size of the axis dimension to the maximum of 0 and the new size of our padded shape

      shape[axis] = tf.minimum(x = shape[axis], y = new_size) # change the size of the axis dimension to the minimum of our original shape and the new size of our padded shape
      shape = tf.stack(values = shape) # stack the list of tensor sizes back into a larger tensor

      resized = tf.concat(values = [
        tf.slice(input_ = tensor, begin = tf.zeros_like(tensor = shape), size = shape), # slice the tensor starting at the 0th index in each dimension and going as far as our adjusted shape in each dimension
        tf.fill(dims = tf.stack(values = pad_shape), value = tf.cast(x = fill_value, dtype = tensor.dtype)) # fill the rest of the tensor with the fill value
      ], axis = axis) # concatenate our sliced tensor with our fill value tensor together

      new_shape = tensor.get_shape().as_list() # get the static shape of the tensor and output it to a list
      new_shape[axis] = new_size # change the static shape's axis to our new size
      resized.set_shape(shape = new_shape) # set the static shape of our resized tensor to our new shape
      return resized # return the resized tensor

    def decode_example(serialized_examples):
      # Create context and sequence feature map
      context_features = {
        "video_id": tf.FixedLenFeature(shape = [], dtype = tf.string),
        "labels": tf.VarLenFeature(dtype = tf.int64)
      }
      sequence_features = {
        "rgb": tf.FixedLenSequenceFeature(shape = [], dtype = tf.string),
        "audio": tf.FixedLenSequenceFeature(shape = [], dtype = tf.string)
      }

      # Parse TF Records into our features
      contexts, features = tf.parse_single_sequence_example(
          serialized = serialized_examples, 
          context_features = context_features,
          sequence_features = sequence_features)
      print("\nread_dataset_frame: _input_fn: decode_example: contexts = {}".format(contexts))
      print("read_dataset_frame: _input_fn: decode_example: features = {}".format(features))
      
      # Create features
      # Pass video_id to features
      features['video_id'] = contexts['video_id']
      print("read_dataset_frame: _input_fn: decode_example: features = {}".format(features))
      
      # Fix rgb data
      decoded_rgb = tf.reshape(tf.cast(tf.decode_raw(features["rgb"], tf.uint8), tf.float32), [-1, 1024])
      print("read_dataset_frame: _input_fn: decode_example: decoded_rgb = {}".format(decoded_rgb))
      rgb_matrix = resize_axis(tensor = dequantize(decoded_rgb), axis = 0, new_size = MAX_FRAMES)
      print("read_dataset_frame: _input_fn: decode_example: rgb_matrix = {}".format(rgb_matrix))
      features['rgb'] = rgb_matrix
      print("read_dataset_frame: _input_fn: decode_example: features = {}".format(features))

      # Fix audio data
      decoded_audio = tf.reshape(tf.cast(tf.decode_raw(features["audio"], tf.uint8), tf.float32), [-1, 128])
      print("read_dataset_frame: _input_fn: decode_example: decoded_audio = {}".format(decoded_audio))
      audio_matrix = resize_axis(tensor = dequantize(decoded_audio), axis = 0, new_size = MAX_FRAMES)
      print("read_dataset_frame: _input_fn: decode_example: audio_matrix = {}".format(audio_matrix))
      features['audio'] = audio_matrix
      print("read_dataset_frame: _input_fn: decode_example: features = {}".format(features))

      # Extract and format labels
      labels = tf.cast(x = tf.sparse_to_dense(sparse_indices = contexts['labels'].values, output_shape = (NUM_CLASSES,), sparse_values = 1, validate_indices = False), dtype = tf.float32)
      print("read_dataset_frame: _input_fn: decode_example: labels = {}\n".format(labels))

      return features, labels

    # Create list of files from file pattern
    file_list = tf.gfile.Glob(filename = file_pattern)
    #print("read_dataset_frame: _input_fn: file_list = {}".format(file_list))

    # Create dataset from file list
    dataset = tf.data.TFRecordDataset(filenames = file_list)
    print("read_dataset_frame: _input_fn: dataset.TFRecordDataset = {}".format(dataset))

    # Decode TF Record dataset examples
    dataset = dataset.map(map_func = lambda x: decode_example(serialized_examples = x))
    print("read_dataset_frame: _input_fn: dataset.map = {}".format(dataset))

    # Determine amount of times to repeat file and if we should shuffle based on if we are training or evaluating
    if mode == tf.estimator.ModeKeys.TRAIN:
      num_epochs = None # read files forever
      
      # Shuffle the dataset within a buffer
      dataset = dataset.shuffle(buffer_size = batch_size * 10, seed = None)
      print("read_dataset_frame: _input_fn: dataset.shuffle = {}".format(dataset))
    else:
      num_epochs = 1 # read files only once

    # Repeat files num_epoch times
    dataset = dataset.repeat(count = num_epochs)
    print("read_dataset_frame: _input_fn: dataset.repeat = {}".format(dataset))

    # Group the data into batches
    dataset = dataset.batch(batch_size = batch_size)
    print("read_dataset_frame: _input_fn: dataset.batch = {}".format(dataset))

    # Create a iterator and then pull the next batch of features and labels from the example queue
    batch_features, batch_labels = dataset.make_one_shot_iterator().get_next()
    print("read_dataset_frame: _input_fn: batch_features = {}".format(batch_features))
    print("read_dataset_frame: _input_fn: batch_labels = {}\n".format(batch_labels))

    return batch_features, batch_labels
  return _input_fn

In [0]:
def try_input_function():
  with tf.Session() as sess:
    fn = read_dataset_frame(file_pattern = "gs://youtube-8m-team/1/frame_level/train/train*.tfrecord", mode = tf.estimator.ModeKeys.TRAIN, batch_size = 2)
    batch_features, batch_labels = fn()
    features, labels = sess.run([batch_features, batch_labels])
    
    print("\ntry_input_function: features = {}".format(features))
    print("try_input_function: labels = {}\n".format(labels))

In [0]:
try_input_function()


read_dataset_frame: _input_fn: file_pattern = gs://youtube-8m-team/1/frame_level/train/train*.tfrecord
read_dataset_frame: _input_fn: mode = train
read_dataset_frame: _input_fn: batch_size = 2
read_dataset_frame: _input_fn: dataset.TFRecordDataset = <TFRecordDataset shapes: (), types: tf.string>

read_dataset_frame: _input_fn: decode_example: contexts = {'labels': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7f8e88710c10>, 'video_id': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:3' shape=() dtype=string>}
read_dataset_frame: _input_fn: decode_example: features = {'rgb': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:5' shape=(?,) dtype=string>, 'audio': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:4' shape=(?,) dtype=string>}
read_dataset_frame: _input_fn: decode_example: features = {'rgb': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:5' shape=(?,) dtype=string>, 'audio': <tf.Tenso

In [0]:
# Create our model function to be used in our custom estimator
def frame_level_model(features, labels, mode, params):
  print("\nframe_level_model: features = {}".format(features)) # features['rgb'].shape = (batch_size, MAX_FRAMES, 1024), features['audio'].shape = (batch_size, MAX_FRAMES, 128)
  print("frame_level_model: labels = {}".format(labels))
  print("frame_level_model: mode = {}".format(mode))

  # 0. Configure network
  # Get dynamic batch size
  current_batch_size = tf.shape(features['rgb'])[0]
  print("frame_level_model: current_batch_size = {}".format(current_batch_size))

  # Stack all of the features into a 3-D tensor
  combined_features = tf.concat(values = [features['rgb'], features['audio']], axis = 2) # shape = (current_batch_size, MAX_FRAMES, 1024 + 128)
  print("frame_level_model: combined_features = {}".format(combined_features))

  # Reshape the combined features into a 2-D tensor that we will pass through our DNN
  reshaped_combined_features = tf.reshape(tensor = combined_features, shape = [current_batch_size * MAX_FRAMES, 1024 + 128]) # shape = (current_batch_size * MAX_FRAMES, 1024 + 128)
  print("frame_level_model: reshaped_combined_features = {}".format(reshaped_combined_features))

  # 1. Create the DNN structure now
  # Create the input layer to our frame DNN
  network = reshaped_combined_features # shape = (current_batch_size * MAX_FRAMES, 1024 + 128)
  print("frame_level_model: network = reshaped_combined_features = {}".format(network))

  # Add hidden layers with the given number of units/neurons per layer
  for units in params['hidden_units']:
    network = tf.layers.dense(inputs = network, units = units, activation = tf.nn.relu) # shape = (current_batch_size * MAX_FRAMES, units)
    print("frame_level_model: network = {}, units = {}".format(network, units))

  # Connect the final hidden layer to a dense layer with no activation to get the logits
  logits = tf.layers.dense(inputs = network, units = NUM_CLASSES, activation = None) # shape = (current_batch_size * MAX_FRAMES, NUM_CLASSES)
  print("frame_level_model: logits = {}".format(logits))

  # Since this is a multi-class, multi-label problem we will apply a sigmoid, not a softmax, to each logit to get its own probability
  probabilities = tf.sigmoid(logits) # shape = (current_batch_size * MAX_FRAMES, NUM_CLASSES)
  print("frame_level_model: probabilities = {}".format(probabilities))

  # Reshape the probabilities back into a 3-D tensor so that we have a frames axis
  reshaped_probabilities = tf.reshape(tensor = probabilities, shape = [current_batch_size, MAX_FRAMES, NUM_CLASSES]) # shape = (current_batch_size, MAX_FRAMES, NUM_CLASSES)
  print("frame_level_model: reshaped_probabilities = {}".format(reshaped_probabilities))

  # Find the average probability over all frames for each label for each example in the batch
  average_probabilities_over_frames = tf.reduce_mean(input_tensor = reshaped_probabilities, axis = 1) # shape = (current_batch_size, NUM_CLASSES)
  print("frame_level_model: average_probabilities_over_frames = {}".format(average_probabilities_over_frames))

  # Select the top k probabilities in descending order
  top_k_probabilities = tf.nn.top_k(input = average_probabilities_over_frames, k = params['top_k'], sorted = True) # shape = (current_batch_size, top_k)
  print("frame_level_model: top_k_probabilities = {}".format(top_k_probabilities))

  # Find the logits from the average probabilities by using the inverse logit 
  inverse_probabilities_logits = tf.log(average_probabilities_over_frames + 0.00000001) - tf.log(1.0 - average_probabilities_over_frames + 0.00000001) # shape = (current_batch_size, NUM_CLASSES)
  print("frame_level_model: inverse_probabilities_logits = {}".format(inverse_probabilities_logits))

  # Select the top k logits using the indices of the top k probabilities in descending order
  top_k_logits = tf.map_fn(fn = lambda x: tf.gather(params = inverse_probabilities_logits[x], indices = top_k_probabilities.indices[x]), 
                                elems = tf.range(start = 0, limit = current_batch_size), 
                                dtype = tf.float32) # shape = (current_batch_size, 1, top_k)
  print("frame_level_model: top_k_logits = {}".format(top_k_logits))

  # Select the top k classes in descending order of likelihood
  top_k_classes = top_k_probabilities.indices # shape = (current_batch_size, top_k)
  print("frame_level_model: top_k_classes = {}".format(top_k_classes))

  # The 0/1 predictions based on a threshold, in this case the threshold is if the probability it greater than random chance
  predictions = tf.where(
    condition = average_probabilities_over_frames > 1.0 / NUM_CLASSES, # shape = (current_batch_size, NUM_CLASSES)
    x = tf.ones_like(tensor = average_probabilities_over_frames), 
    y = tf.zeros_like(tensor = average_probabilities_over_frames))
  print("frame_level_model: predictions = {}".format(predictions))

  # The 0/1 top k predictions based on a threshold, in this case the threshold is if the probability it greater than random chance
  top_k_predictions = tf.where(
    condition = top_k_probabilities.values > 1.0 / NUM_CLASSES, # shape = (current_batch_size, top_k)
    x = tf.ones_like(tensor = top_k_probabilities.values), 
    y = tf.zeros_like(tensor = top_k_probabilities.values))
  print("frame_level_model: top_k_predictions = {}\n".format(top_k_predictions))

  # 2. Loss function, training/eval ops 
  if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
    # Since this is a multi-class, multi-label problem, we will use sigmoid activation and cross entropy loss
    # We already have the probabilities we can use the cross entropy formula directly to calculate the loss
    loss = tf.reduce_mean(input_tensor = -tf.reduce_sum(input_tensor = labels * tf.log(x = average_probabilities_over_frames + 0.00000001), axis = 1))

    train_op = tf.contrib.layers.optimize_loss(
      loss = loss,
      global_step = tf.train.get_global_step(),
      learning_rate = 0.01,
      optimizer = "Adam")
    eval_metric_ops = {
        "accuracy": tf.metrics.mean_per_class_accuracy(labels = labels, predictions = predictions, num_classes = NUM_CLASSES)
    }
  else:
    loss = None
    train_op = None
    eval_metric_ops = None

  # 3. Create predictions
  predictions_dict = {"logits": top_k_logits, 
                      "probabilities": top_k_probabilities.values, 
                      "predictions": top_k_predictions,
                      "classes": top_k_classes}

  # 4. Create export outputs
  export_outputs = {"predict_export_outputs": tf.estimator.export.PredictOutput(outputs = predictions_dict)}

  # 5. Return EstimatorSpec
  return tf.estimator.EstimatorSpec(
    mode = mode,
    predictions = predictions_dict,
    loss = loss,
    train_op = train_op,
    eval_metric_ops = eval_metric_ops,
    export_outputs = export_outputs)

In [0]:
# Create our serving input function to accept the data at serving and send it in the right format to our custom estimator
def serving_input_fn():
  # This function fixes the shape and type of our input strings
  def fix_shape_and_type_for_serving(placeholder):
    # String split each string in the batch and output the values from the resulting SparseTensors
    split_string = tf.map_fn(
      fn = lambda x: tf.string_split(source = [placeholder[x]], delimiter=',').values, 
      elems = tf.range(start = 0, limit = tf.shape(input = placeholder)[0]), 
      dtype = tf.string) # shape = (batch_size, input_sequence_length)
    print("serving_input_fn: fix_shape_and_type_for_serving: split_string = {}".format(split_string))

    # Convert each string in the split tensor to float
    feature_tensor = tf.string_to_number(string_tensor = split_string, out_type = tf.float32) # shape = (batch_size, input_sequence_length)
    print("serving_input_fn: fix_shape_and_type_for_serving: feature_tensor = {}".format(feature_tensor))
    return feature_tensor
  
  # This function fixes dynamic shape ambiguity of last dimension so that we will be able to use it in our DNN (since tf.layers.dense require the last dimension to be known)
  def get_shape_and_set_modified_shape_3D(tensor, additional_dimension_sizes):
    # Get static shape for tensor and convert it to list
    shape = tensor.get_shape().as_list()
    # Set outer shape to additional_dimension_sizes[0] * additional_dimension_sizes[1] since we know that this is the correct size
    shape[1] = additional_dimension_sizes[0] * additional_dimension_sizes[0]
    # Set the shape of tensor to our modified shape
    tensor.set_shape(shape = shape) # shape = (batch_size, additional_dimension_sizes[0] * additional_dimension_sizes[1])
    print("serving_input_fn: get_shape_and_set_modified_shape_3D: tensor = {}, additional_dimension_sizes = {}".format(tensor, additional_dimension_sizes))
    # Finally reshape tensor into the shape it is supposed to be for the model function
    tensor = tf.reshape(tensor = tensor, shape = [-1, additional_dimension_sizes[0], additional_dimension_sizes[1]]) # shape = (batch_size, additional_dimension_sizes[0], additional_dimension_sizes[1])
    print("serving_input_fn: get_shape_and_set_modified_shape_3D: tensor = {}, additional_dimension_sizes = {}".format(tensor, additional_dimension_sizes))
    return tensor
  
  # Create placeholders to accept the data sent to the model at serving time
  feature_placeholders = { # all features come in as a batch of strings, shape = (batch_size,), this was so because of passing the arrays to online ml-engine prediction
    'video_id': tf.placeholder(dtype = tf.string, shape = [None]),
    'rgb': tf.placeholder(dtype = tf.string, shape = [None]),
    'audio': tf.placeholder(dtype = tf.string, shape = [None])
  }
  print("\nserving_input_fn: feature_placeholders = {}".format(feature_placeholders))

  # Create feature tensors
  features = {
    "video_id": feature_placeholders["video_id"],
    "rgb": fix_shape_and_type_for_serving(placeholder = feature_placeholders["rgb"]),
    "audio": fix_shape_and_type_for_serving(placeholder = feature_placeholders["audio"])
  }
  print("serving_input_fn: features = {}".format(features))

  # Fix dynamic shape ambiguity of feature tensors for our DNN
  features["rgb"] = get_shape_and_set_modified_shape_3D(tensor = features["rgb"], additional_dimension_sizes = [MAX_FRAMES, 1024])
  features["audio"] = get_shape_and_set_modified_shape_3D(tensor = features["audio"], additional_dimension_sizes = [MAX_FRAMES, 128])
  print("serving_input_fn: features = {}\n".format(features))
  
  return tf.estimator.export.ServingInputReceiver(features = features, receiver_tensors = feature_placeholders)

In [0]:
# Create custom estimator's train and evaluate function
def train_and_evaluate(args):
  # Create our custome estimator using our model function
  estimator = tf.estimator.Estimator(
    model_fn = frame_level_model, 
    model_dir = args['output_dir'],
    params = {'hidden_units': args['hidden_units'], 'top_k': args['top_k']})
  # Create train spec to read in our training data
  train_spec = tf.estimator.TrainSpec(
    input_fn = read_dataset_frame(
      file_pattern = args['train_file_pattern'], 
      mode = tf.estimator.ModeKeys.TRAIN, 
      batch_size = args['batch_size']),
    max_steps = args['train_steps'])
  # Create exporter to save out the complete model to disk
  exporter = tf.estimator.LatestExporter(name = 'exporter', serving_input_receiver_fn = serving_input_fn)
  # Create eval spec to read in our validation data and export our model
  eval_spec = tf.estimator.EvalSpec(
    input_fn = read_dataset_frame(
      file_pattern = args['eval_file_pattern'], 
      mode = tf.estimator.ModeKeys.EVAL, 
      batch_size = args['batch_size']),
    steps = None,
    exporters = exporter,
    start_delay_secs = args['start_delay_secs'],
    throttle_secs = args['throttle_secs'])
  # Create train and evaluate loop to train and evaluate our estimator
  tf.estimator.train_and_evaluate(estimator = estimator, train_spec = train_spec, eval_spec = eval_spec)

In [0]:
# Run the training job
shutil.rmtree(arguments['output_dir'], ignore_errors = True) # start fresh each time
train_and_evaluate(args = arguments)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc22b1f1690>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained_model', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 30 secs (eval_spec.throttle_secs) or training is finished.

read_dataset_frame: _input_fn: file_pattern = gs://youtube-8m-team/1/frame_level/train/train*.tfrecord
read_dataset_frame: _input_

# Training

### Locally

In [0]:
%bash
OUTDIR=trained_model
rm -rf $OUTDIR
export PYTHONPATH=$PYTHONPATH:$PWD/trainer
python -m trainer.task \
  --train_file_pattern="gs://youtube-8m-team/1/frame_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/frame_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --batch_size=10 \
  --train_steps=100 \
  --hidden_units="1024 512 256" \
  --top_k=5 \
  --job-dir=./tmp


read_dataset_frame: _input_fn: file_pattern = gs://youtube-8m-team/1/frame_level/train/train*.tfrecord
read_dataset_frame: _input_fn: mode = train
read_dataset_frame: _input_fn: batch_size = 10
read_dataset_frame: _input_fn: dataset.TFRecordDataset = <TFRecordDataset shapes: (), types: tf.string>
read_dataset_frame: _input_fn: decode_example: contexts = {'labels': <tensorflow.python.framework.sparse_tensor.SparseTensor object at 0x7fae90d85e90>, 'video_id': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:3' shape=() dtype=string>}
read_dataset_frame: _input_fn: decode_example: features = {'rgb': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:5' shape=(?,) dtype=string>, 'audio': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:4' shape=(?,) dtype=string>}
read_dataset_frame: _input_fn: decode_example: features = {'rgb': <tf.Tensor 'ParseSingleSequenceExample/ParseSingleSequenceExample:5' shape=(?,) dtype=string>, 'audio': <tf.Tenso

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fae91e8afd0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': 'trained_model/', '_global_id_in_cluster': 0, '_save_summary_steps': 100}
INFO:tensorflow:Running tr

### GCloud

In [0]:
%bash
OUTDIR=gs://$BUCKET/youtube_8m_frame_level_datasets/trained_model
JOBNAME=job_youtube_8m_frame_level_datasets$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --runtime-version=1.5 \
  -- \
  --train_file_pattern="gs://youtube-8m-team/1/frame_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/frame_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --batch_size=50 \
  --train_steps=10000 \
  --hidden_units="1024 512 256" \
  --top_k=5 \
  --job-dir=$OUTDIR

gs://youtube8m-4-train/youtube_8m_frame_level_datasets/trained_model us-central1 job_youtube_8m_frame_level_datasets180525_053143
jobId: job_youtube_8m_frame_level_datasets180525_053143
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [job_youtube_8m_frame_level_datasets180525_053143] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe job_youtube_8m_frame_level_datasets180525_053143

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs job_youtube_8m_frame_level_datasets180525_053143


### Hyperparameter tuning

In [0]:
%writefile hyperparam.yaml
trainingInput:
  scaleTier: STANDARD_1
  hyperparameters:
    hyperparameterMetricTag: accuracy
    goal: MAXIMIZE
    maxTrials: 30
    maxParallelTrials: 1
    params:
    - parameterName: batch_size
      type: INTEGER
      minValue: 8
      maxValue: 512
      scaleType: UNIT_LOG_SCALE
    - parameterName: hidden_units
      type: CATEGORICAL
      categoricalValues: ["64 32", "256 128 16", "64 64 64 8"]

Writing hyperparam.yaml


In [0]:
%bash
OUTDIR=gs://$BUCKET/youtube_8m_frame_level_datasets/hyperparam
JOBNAME=job_youtube_8m_frame_level_datasets$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
  --region=$REGION \
  --module-name=trainer.task \
  --package-path=$PWD/trainer \
  --job-dir=$OUTDIR \
  --staging-bucket=gs://$BUCKET \
  --scale-tier=STANDARD_1 \
  --config=hyperparam.yaml \
  --runtime-version=1.5 \
  -- \
  --train_file_pattern="gs://youtube-8m-team/1/frame_level/train/train*.tfrecord" \
  --eval_file_pattern="gs://youtube-8m-team/1/frame_level/validate/validate-0.tfrecord"  \
  --output_dir=$OUTDIR \
  --train_steps=10000 \
  --top_k=5 \
  --job-dir=$OUTDIR

gs://youtube8m-4-train/youtube_8m_frame_level_datasets/hyperparam us-central1 job_youtube_8m_frame_level_datasets180525_061627
jobId: job_youtube_8m_frame_level_datasets180525_061627
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [job_youtube_8m_frame_level_datasets180525_061627] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe job_youtube_8m_frame_level_datasets180525_061627

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs job_youtube_8m_frame_level_datasets180525_061627


# Deploy

In [0]:
%bash
MODEL_NAME="youtube_8m_frame_level_datasets"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://$BUCKET/youtube_8m_frame_level_datasets/trained_model/export/exporter/ | tail -1)
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION ... this will take a few minutes"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create $MODEL_NAME --regions $REGION
gcloud ml-engine versions create $MODEL_VERSION --model $MODEL_NAME --origin $MODEL_LOCATION --runtime-version 1.5

Deleting and deploying youtube_8m_frame_level_datasets v1 from gs://youtube8m-4-train/youtube_8m_frame_level_datasets/trained_model/export/exporter/1527219410/ ... this will take a few minutes


Created ml engine model [projects/qwiklabs-gcp-8d3d0cd07cef9252/models/youtube_8m_frame_level_datasets].
Creating version (this might take a few minutes)......
.............................................................................................................done.


# Prediction

### Prep

In [0]:
# Let's call our input function to decode our data to put into BigQuery for testing predictions
video_list = []
with tf.Session() as sess:
  fn = read_dataset_frame(
    file_pattern = "gs://youtube-8m-team/1/frame_level/validate/validate-0.tfrecord", 
    mode = tf.estimator.ModeKeys.EVAL, 
    batch_size = 1)
  batch_features, batch_labels = fn()
  for key,value in batch_features.items():
    batch_features[key] = tf.squeeze(batch_features[key])
  fixed_batch_features = batch_features
  fixed_batch_labels = tf.squeeze(batch_labels)

  while True:
    features, labels = sess.run([fixed_batch_features, fixed_batch_labels])
    features["labels"] = labels
    video_list.append(features)

In [0]:
# This is the number of videos from the frame level file we just processed
len(video_list)

331

In [0]:
# Convert the nd-arrays to lists and cast to strings (video_id is already a single string)
for items in video_list:
  items["rgb"] = str(items["rgb"].tolist())
  items["audio"] = str(items["audio"].tolist())
  items["labels"] = str(items["labels"].tolist())

In [0]:
# Create a dataframe from the list
video_df = pd.DataFrame(video_list)
video_df = video_df[["video_id", "rgb", "audio", "labels"]]
video_df.head()

,video_id,rgb,audio,labels
0,-0AjrnQ0N3A,"[[-1.9921875, -0.8627756834030151, 0.721538066...","[[0.7215380668640137, -1.5686581134796143, -0....","[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,-0MpgkddrY4,"[[0.0313420295715332, -0.28238344192504883, 1....","[[1.3019304275512695, 0.925459623336792, 1.458...","[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,-02RMEBtLDo,"[[-0.7686580419540405, -0.47061872482299805, 2...","[[1.082322597503662, 1.7568323612213135, 1.929...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,-00LLAtj0JE,"[[-1.9921875, 0.6587929725646973, -0.752971768...","[[0.7215380668640137, -1.5686581134796143, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,-07wapPiIAg,"[[-1.1921874284744263, 1.1450676918029785, 0.8...","[[0.7215380668640137, -1.2235599756240845, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [0]:
# Export dataframe to BigQuery
import datalab.bigquery as bq
bigquery_dataset_name = 'ryan_youtube'
bigquery_table_name = 'tbl_frame_level'

# Define BigQuery dataset and table\n",
dataset = bq.Dataset(bigquery_dataset_name)
table = bq.Table(bigquery_dataset_name + '.' + bigquery_table_name)

# Create BigQuery dataset
if not dataset.exists():
    dataset.create()

# Create or overwrite the existing table if it exists\n",
table_schema = bq.Schema.from_dataframe(video_df)
table.create(schema = table_schema, overwrite = True)

video_df.to_gbq(destination_table = bigquery_dataset_name + '.' + bigquery_table_name, project_id = "qwiklabs-gcp-8d3d0cd07cef9252", if_exists = "replace")




Load is 100% Complete




In [0]:
# Create SQL query
query="""
SELECT
  video_id,
  rgb,
  audio
FROM
  `qwiklabs-gcp-8d3d0cd07cef9252.ryan_youtube.tbl_frame_level`
LIMIT
  3
"""

In [0]:
# Export BigQuery results to dataframe
import google.datalab.bigquery as bq2
df_predict = bq2.Query(query).execute().result().to_dataframe()
df_predict.head()

,video_id,rgb,audio
0,-0FDy3F9Gqo,"[[-0.21963834762573242, 1.1293814182281494, 0....","[[-0.7372854948043823, 0.42349886894226074, -0..."
1,-0CK7kXtP-U,"[[-1.9921875, -0.8627756834030151, 0.721538066...","[[-0.8314031362533569, -0.21963834762573242, -..."
2,-0QTVHnvT90,"[[-1.9921875, -0.5804227590560913, 0.439185142...","[[0.5803616046905518, -0.7215992212295532, 0.5..."


### Local prediction from local model

In [0]:
# Format dataframe to new line delimited json strings and write out to json file
with open('frame_level.json', 'w') as outfile:
  for idx, row in df_predict.iterrows():
    json_string = "{\"video_id\": \"" + row["video_id"] + "\", \"rgb\": \"" + row["rgb"].replace(" ","").replace("[","").replace("]","") + "\", \"audio\": \"" + row["audio"].replace(" ","").replace("[","").replace("]","") + "\"}"
    outfile.write("%s\n" % json_string)

In [0]:
%bash
model_dir=$(ls $PWD/trained_model/export/exporter | tail -1)
gcloud ml-engine local predict \
    --model-dir=$PWD/trained_model/export/exporter/$model_dir \
    --json-instances=./frame_level.json

CLASSES             LOGITS                                                                                             PREDICTIONS                PROBABILITIES
[0, 9, 2, 43, 15]   [6.291934967041016, 6.258187294006348, 6.062037944793701, 5.9421892166137695, 5.871412754058838]   [1.0, 1.0, 1.0, 1.0, 1.0]  [0.9981522560119629, 0.9980889558792114, 0.9976757764816284, 0.9973806142807007, 0.9971890449523926]
[9, 43, 0, 15, 19]  [8.108847618103027, 7.53411865234375, 7.450527191162109, 7.245649814605713, 7.1199798583984375]    [1.0, 1.0, 1.0, 1.0, 1.0]  [0.9996992349624634, 0.9994657635688782, 0.9994192123413086, 0.9992872476577759, 0.9991918802261353]
[9, 0, 2, 43, 15]   [10.908120155334473, 10.618941307067871, 10.082742691040039, 9.946921348571777, 9.76628589630127]  [1.0, 1.0, 1.0, 1.0, 1.0]  [0.999981701374054, 0.9999755620956421, 0.9999582171440125, 0.9999521374702454, 0.9999426603317261]


  from ._conv import register_converters as _register_converters
2018-05-25 05:30:01.507851: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



### GCloud ML-Engine prediction from deployed model

In [0]:
# Format dataframe to instances list to get sent to ML-Engine
instances = [{"video_id": row["video_id"], "rgb": row["rgb"].replace(" ","").replace("[","").replace("]",""), "audio": row["audio"].replace(" ","").replace("[","").replace("]","")} for idx, row in df_predict.iterrows()]

In [0]:
# Send instance dictionary to receive response from ML-Engine for online prediction
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials)

request_data = {"instances": instances}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'youtube_8m_frame_level_datasets', 'v1')
response = api.projects().predict(body = request_data, name = parent).execute()
print("response = {}".format(response))

response = {u'predictions': [{u'probabilities': [1.0, 1.0, 1.0, 1.0, 1.0], u'logits': [inf, inf, inf, inf, inf], u'classes': [0, 1, 2, 3, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}, {u'probabilities': [1.0, 1.0, 1.0, 1.0, 1.0], u'logits': [inf, inf, inf, inf, inf], u'classes': [0, 1, 2, 3, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}, {u'probabilities': [1.0, 1.0, 1.0, 1.0, 1.0], u'logits': [inf, inf, inf, inf, inf], u'classes': [0, 1, 2, 3, 4], u'predictions': [1.0, 1.0, 1.0, 1.0, 1.0]}]}
